In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
%cd /content/drive/MyDrive/HRP/

In [ ]:
!ls

In [ ]:
!pip install gpt4all[cuda]

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from gpt4all import GPT4All
from pathlib import Path
from tqdm import tqdm

In [ ]:
model_dir = Path(r"./")
model_dir.mkdir(exist_ok=True)

model = GPT4All(
    model_name="Meta-Llama-3-8B-Instruct.Q4_0.gguf",
    model_path=model_dir,
    allow_download=True,
    n_threads=4,
    device="cuda", # lets use that gpu
    ngl=100
)

In [ ]:
def make_prompt(text):
  prompt = (
      "You are a medical expert. Extract dosages for all drugs: "
      "Return ONLY the following list [(drug,dosage,unit,frequency)] for each drug return nothing else at all"
      f"{text}\nAnswer:"
  )
  return prompt

def extract_with_gpt4all(row, max_tokens=256):
    prompt = make_prompt(row["note_text"])

    response = model.generate(prompt, n_predict=max_tokens)
    return response.strip()

In [ ]:
csv_path = "unique_pharmacy_notes.csv"
df = pd.read_csv(csv_path)
df.head()

In [ ]:
tqdm.pandas(desc="Extracting with GPT4All")
df["extraction"] = df.progress_apply(extract_with_gpt4all, axis=1)

df.to_csv("extracted_pharmacy_notes.csv", index=False)

In [ ]:
df.head()